In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam

# Load data
data = pd.read_csv('data.csv')

# Compute delta health
data['delta_p1_health'] = data['p1Health'] - data['p1Health'].shift(1)
data['delta_p2_health'] = data['p2Health'] - data['p2Health'].shift(1)

# Compute delta distance
data['delta_distance'] = ((data['p1Xcoord'] - data['p2Xcoord']) * 2 + (data['p1Ycoord'] - data['p2Ycoord']) * 2) * 0.5 - ((data['p1Xcoord'].shift(1) - data['p2Xcoord'].shift(1)) * 2 + (data['p1Ycoord'].shift(1) - data['p2Ycoord'].shift(1)) * 2) * 0.5

# Drop the first row since it has missing values due to shift operation
data = data.dropna()

# Convert categorical variables to numerical values
data['isJumpP1'] = data['isJumpP1'].astype(int)
data['isCrouchP1'] = data['isCrouchP1'].astype(int)
data['isJumpP2'] = data['isJumpP2'].astype(int)
data['isCrouchP2'] = data['isCrouchP2'].astype(int)
data['FightResult'] = pd.factorize(data['FightResult'])[0]

# Select features and target variable
X = data[['p1Xcoord', 'p1Health', 'p1Ycoord', 'isJumpP1', 'isCrouchP1', 'p2Xcoord', 'p2Health', 'p2Ycoord', 'isJumpP2', 'isCrouchP2', 'p1PlayerId', 'P1InMove', 'P1MoveId', 'p2PlayerId', 'P2InMove', 'P2MoveId', 'timer', 'RoundStart', 'RoundOver', 'delta_p1_health', 'delta_p2_health', 'delta_distance']]
y = data[[ 'P1ButtonsUp', 'P1ButtonsDown', 'P1ButtonsRight', 'P1ButtonsLeft', 'P1ButtonsY', 'P1ButtonsB', 'P1ButtonsX', 'P1ButtonsA', 'P1ButtonsL', 'P1ButtonsR']]

# Select the data where delta_p2_health is greater than 0
data_filtered = data[data['delta_p2_health'] > 0]

# Print the filtered data
print(data_filtered)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Normalize continuous variables
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Define the model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train.values, batch_size=32, epochs=50, validation_split=0.2, verbose=1)

# Evaluate the model
score = model.evaluate(X_test, y_test.values)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('model.h5')

       p1Xcoord  p1Health  p1Ycoord  isJumpP1  isCrouchP1  P1ButtonsUp   
2908        244        42       192         0           0        False  \
3791        205       176       192         0           0        False   
4501        205       176       192         0           0        False   
6636        284       176       192         0           0        False   
7663        209        91       192         0           0        False   
10194       205       176       192         0           0        False   
13976       205       176       192         0           0        False   
16901       205       176       192         0           0        False   
17876       205       176       192         0           0        False   

       P1ButtonsDown  P1ButtonsRight  P1ButtonsLeft  P1ButtonsY  ...   
2908           False           False          False       False  ...  \
3791           False           False          False       False  ...   
4501           False           False       

c:\Python39\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


371/371 [==============================] - 2s 4ms/step - loss: 1.6013 - accuracy: 0.1804 - val_loss: 1.4075 - val_accuracy: 0.2756
Epoch 2/50
371/371 [==============================] - 1s 4ms/step - loss: 1.4417 - accuracy: 0.2072 - val_loss: 1.3727 - val_accuracy: 0.2753
Epoch 3/50
371/371 [==============================] - 1s 3ms/step - loss: 1.4792 - accuracy: 0.1996 - val_loss: 1.3973 - val_accuracy: 0.1471
Epoch 4/50
371/371 [==============================] - 1s 3ms/step - loss: 1.6243 - accuracy: 0.1776 - val_loss: 1.6647 - val_accuracy: 0.1039
Epoch 5/50
371/371 [==============================] - 1s 3ms/step - loss: 1.8353 - accuracy: 0.1717 - val_loss: 1.5344 - val_accuracy: 0.1289
Epoch 6/50
371/371 [==============================] - 1s 3ms/step - loss: 2.0175 - accuracy: 0.1690 - val_loss: 1.8131 - val_accuracy: 0.0951
Epoch 7/50
371/371 [==============================] - 1s 3ms/step - loss: 2.4339 - accuracy: 0.1689 - val_loss: 1.6652 - val_accuracy: 0.2770
Epoch 8/50
371/37

In [67]:
#X = data[['p1Xcoord', 'p1Health', 'p1Ycoord', 'isJumpP1', 'isCrouchP1', 'p2Xcoord', 'p2Health', 'p2Ycoord', 'isJumpP2', 'isCrouchP2', 'p1PlayerId', 'P1InMove', 'P1MoveId', 'p2PlayerId', 'P2InMove', 'P2MoveId', 'timer', 'RoundStart', 'RoundOver', 'delta_p1_health', 'delta_p2_health', 'delta_distance']]

y_pred = model.predict(X_train)

#write y_pred to csv
np.savetxt("y_pred.csv", y_pred, delimiter=",")


464/464 [==============================] - 1s 1ms/step


In [58]:
#load model
from keras.models import load_model
model = load_model('model.h5')

#predict
y_pred = model.predict(X_test)

print(y_pred), X_test

52/52 [==============================] - 0s 2ms/step
[[1.07006435e-05 1.19212538e-01 1.98060237e-02 ... 4.45808458e-23
  2.42602060e-30 1.28339481e-04]
 [1.55255534e-02 2.60869563e-01 1.80699065e-01 ... 1.65371750e-06
  1.04384974e-07 4.77915909e-03]
 [2.47171467e-16 1.81540020e-03 3.26659688e-06 ... 0.00000000e+00
  0.00000000e+00 1.91170468e-10]
 ...
 [2.17328935e-14 4.40597720e-03 1.53414949e-05 ... 0.00000000e+00
  0.00000000e+00 1.15734544e-09]
 [1.51675601e-13 6.10245811e-03 3.09165225e-05 ... 0.00000000e+00
  0.00000000e+00 3.58423868e-09]
 [4.53544605e-14 4.64155758e-03 2.19186204e-05 ... 0.00000000e+00
  0.00000000e+00 4.20594137e-09]]


(None,
 array([[-1.87670712e+00, -1.24692628e+00,  2.03315539e-01, ...,
         -9.22819756e-03, -1.29623528e-03, -2.56155943e-03],
        [-5.80037916e-01, -1.65438066e+00,  3.82182487e-01, ...,
         -9.22819756e-03, -1.29623528e-03, -1.89403290e+00],
        [-5.12990188e-02,  1.14686823e+00,  3.82182487e-01, ...,
         -9.22819756e-03, -1.29623528e-03, -2.56155943e-03],
        ...,
        [ 3.68241307e-02,  7.05459311e-01,  3.82182487e-01, ...,
         -9.22819756e-03, -1.29623528e-03, -2.56155943e-03],
        [-5.12990188e-02,  7.05459311e-01,  3.82182487e-01, ...,
         -9.22819756e-03, -1.29623528e-03, -2.56155943e-03],
        [ 2.42351093e-02, -2.28290319e-01, -8.10263831e-01, ...,
         -9.22819756e-03, -1.29623528e-03,  1.76802102e-01]]))